In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import csv
import numpy as np
import SD2_Model as SD

In [ ]:

#net = SD.Net()
device = torch.device("cuda") 
net = SD.Net().to(device)
summary(net, input_size = (1,2))
#print(net)

loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
#dummy data
#trainset = (torch.rand(2,1, device="cuda"), torch.rand(2,1, device="cuda") ) #Xs and Ys
trainset = torch.tensor([[1,2],[3,4]]).to(device)
print(trainset)

In [ ]:
#data = np.loadtxt('../Datasets/2NN_all_dataset.csv' ,delimiter=',', dtype=np.float32)
#print(data.shape[0])

class Coordinates_Dataset(Dataset):
    def __init__(self):
        #load data
        data = np.loadtxt('../Datasets/2NN_all_dataset.csv' ,delimiter=',', dtype=np.float32)
        #feature, data_label = data
        self.feature = torch.from_numpy(data[:,0:2]).to(device)
        self.data_label = torch.from_numpy(data[:,2:4] * 100).to(device) #convert meters to cm
        self.num_samples = data.shape[0]

    def __getitem__(self,index):
        #get data
        #print("index: ",index)
        return self.feature[index],self.data_label[index]


    def __len__(self):
        #len(data)
        return self.num_samples

dataset = Coordinates_Dataset()
feature, label = dataset[0]
print(feature, label)
trainset = DataLoader(dataset=dataset, batch_size=200, shuffle=True)
#data_iter = iter(trainset)
#print(type(data_iter))
#data = data_iter.next()
#print(data)

## Old data loader
Load_data = False
dataset_path = "../Datasets/2NN_dataset_flight.3.csv"
if(Load_data):
    trainset = []
    with open(dataset_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            #feature, data_label = int(row[0:2]),row[2:4]
            feature, data_label = (int(row[0]),int(row[1])),(float(row[2]) * 100 ,float(row[3]) * 100) #convert to cm
            #feature, data_label = (int(row[0]),int(row[1])),(round(float(row[2]) * 100) ,round(float(row[3]) * 100)) #convert to cm
            trainset.append([feature, data_label])
            #print([feature, data_label])
    trainset = torch.Tensor(trainset).to(device)
    #print(trainset)

test_data = False
if(test_data):
    feature, data_label = trainset[0]
    print(feature, data_label)
    #print(feature.view(1,2))
    output = net(feature)
    print(output)


In [ ]:
train = True
epochs = 500
best_loss = 0.0
if (train):
    net.train()
    for epoch in range(epochs):
        for data in trainset:  
            feature, data_label = data #
            #print(data)
            #print(feature)
            #print(data_label)
            net.zero_grad()  
            output = net(feature)
            loss = loss_function(output, data_label) 
            loss.backward() 
            optimizer.step() 
        print(epoch,loss)
        if(epoch == 1):
            best_loss = loss.item()
        if(False):
        #if(loss.item() < best_loss or loss.item() < 1.0):
            best_loss = loss.item()
            best_save = "./models_saves/best_2nd_NN_"
            torch.save(net, best_save + str(epoch) + "_" + str(round(best_loss,5)) +".pt")
    print(best_loss)

In [ ]:
#save network
save = True
save = False
model_save_path = "2nd_NN.pt"
if (save):
    torch.save(net, model_save_path)

In [ ]:
#test evaluate

Load_eval = False
if(Load_eval):
    #model = SD.Net().to(device)
    net = torch.load(model_save_path)
    net.eval()

dataset_path = "../2NN_dataset_flight.2.csv"
if(False):
    net.eval()
    eval_data = []
    with open(dataset_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            #feature, data_label = int(row[0:2]),row[2:4]
            feature, data_label = (int(row[0]),int(row[1])),(float(row[2]) * 100 ,float(row[3]) * 100) #convert to cm
            #feature, data_label = (int(row[0]),int(row[1])),(round(float(row[2]) * 100) ,round(float(row[3]) * 100)) #convert to cm
            net(feature)
            eval_data.append([feature, data_label])
            


